## Objective- join tables togeather with query and clean for EDA 

#Steps - 
1 - Notebook Querys joined data from BQ tables 

2 - nulls and duplicates are taken care off 
3- save cleaned data to drive 

# Next Steps - 
- add shooting data and EDA 


In [ ]:
import pandas as pd

import pandas_gbq
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline



from google.colab import auth
auth.authenticate_user()
print('Authenticated.')

project_id = 'san-jose-earthquakes-dev'
# bigQueryDB = 'san-jose-earthquakes-dev.team_stats'

from google.cloud import  bigquery
client = bigquery.Client(project_id)

Authenticated.


In [ ]:
import sys
!{sys.executable} -m pip install -U pandas-profiling[notebook]
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
import tensorflow_data_validation as tfdv
 


# query to get all merged features from fb_ref tables 

In [ ]:
sql = """
WITH
  -- opposing team stats
  opp AS (
  SELECT
    Squad AS Squad_from_op,
    Opposing_Poss,
    Opposing_Touches
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.opposing_possesion` ),
  -- Mich with aerial stats
  mish AS (
  SELECT
    Player as mish_player,
    Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    Interceptions as def_interceptions,
    Tackles_won as from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won,
     Aerial_Duels_lost,
    Aerial_win_pct 
    from 
    `san-jose-earthquakes-dev.MLS_2021_fbref.Miscellaneous_stats` ),
  -- defensive stats
  def AS (
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.Defensive_stats` ),
  -- player passing stats
  pass AS (
  SELECT
    Player,
    fb_id,
    player_name,
    Nineties,
    CAST(xA AS float64) AS xA,
    CAST(Key_Pass AS int64) AS Key_pass,
    CAST(Comp_prog AS float64) AS Comp_prog,
    Total_Cmp,
    Total_Att,
    Total_Cmp_pct,
    Total_Tot_Dist,
    Total_Prg_Dist,
    Short_Cmp,
    Short_Att,
    Short_Cmp_pct,
    Medium_Cmp,
    Medium_Att,
    Medium_Cmp_pct,
    Long_Cmp,
    Long_Att,
    Long_Cmp_pct,
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.passing_stats` ),
  --player possesion stats
  posses AS(
  SELECT
    *
  FROM
    `san-jose-earthquakes-dev.MLS_2021_fbref.possesion_stats` )
  --
  ### main query below
  --
SELECT
  pass.fb_id,
  pass.player_name,
  opp.Squad_from_op,
  opp.Opposing_Poss,
  opp.Opposing_Touches,
  -- def stats
  def.Nation,
  def.Position,
  pass.Nineties,
  pass.xA,
  pass.Key_pass,
  pass.Comp_prog,
  pass.Total_Cmp,
  pass.Total_Att,
  pass.Total_Cmp_pct,
  pass.Total_Tot_Dist,
  pass.Total_Prg_Dist,
  pass.Short_Cmp,
  pass.Short_Att,
  pass.Short_Cmp_pct,
  pass.Medium_Cmp,
  pass.Medium_Att,
  pass.Medium_Cmp_pct,
  pass.Long_Cmp,
  pass.Long_Att,
  pass.Long_Cmp_pct,
  -- posses below
  posses.touches,
  posses.touches_def_box,
  posses.touches_def_3rd,
  posses.touches_mid_3rd,
  posses.touches_att_3rd,
  posses.touches_att_box,
  posses.touch_live,
  posses.dribble_Succ,
  posses.dribble_Att,
  posses.dribble_Succ_pct,
  posses.num_dribble_past,
  posses.dribble_megs,
  posses.carries,
  posses.carries_dist,
  posses.carries_prog_dist,
  posses.carries_prog,
  posses.carries_att_3rd,
  posses.carries_att_box,
  posses.miss_control,
  posses.dispossed,
  posses.recep_targ,
  posses.recep_succ,
  posses.recept_pct,
  posses.prog_pass_rec

  -- mish

  Yellow_cards,
    Red_cards,
    Second_yellow,
    Fouls,
    Fouls_drawn,
    Offsides,
    Crosses,
    def_interceptions,
    from_mish_tackles_won,
    PK_won,
    PK_conceded,
    OG,
    Recoveries,
    Aerial_Duels_Won Aerial_Duels_lost,
    Aerial_win_pct 
FROM
  def
LEFT JOIN
  pass
ON
  def.Player = pass.Player
left join 
  mish 
on 
  def.Player = mish.mish_player
LEFT JOIN
  posses
ON
  def.Player = posses.Player
LEFT JOIN
  opp
ON
  def.Team = opp.Squad_from_op
ORDER BY
  Key_pass DESC

"""
df = pandas_gbq.read_gbq(sql,project_id=project_id)

Downloading: 100%|██████████| 925/925 [00:00<00:00, 1302.00rows/s]


In [ ]:
info= pd.DataFrame({"values":{col:df[col].unique() for col in df},
              'type':{col:df[col].dtype for col in df},
              'unique values':{col:len(df[col].unique()) for col in df},
              'NA values':{col:str(round(sum(df[col].isna())/len(df),2))+'%' for col in df},
              'Duplicated Values':{col:sum(df[col].duplicated()) for col in df}
             })

info.T

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Nation,Position,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,Total_Tot_Dist,Total_Prg_Dist,Short_Cmp,Short_Att,Short_Cmp_pct,Medium_Cmp,Medium_Att,Medium_Cmp_pct,Long_Cmp,Long_Att,Long_Cmp_pct,touches,touches_def_box,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_box,touch_live,dribble_Succ,dribble_Att,dribble_Succ_pct,num_dribble_past,dribble_megs,carries,carries_dist,carries_prog_dist,carries_prog,carries_att_3rd,carries_att_box,miss_control,dispossed,recep_targ,recep_succ,recept_pct,Yellow_cards,Red_cards,Second_yellow,Fouls,Fouls_drawn,Offsides,Crosses,def_interceptions,from_mish_tackles_won,PK_won,PK_conceded,OG,Recoveries,Aerial_Duels_lost,Aerial_win_pct
values,"[11c30a46, acd47bc0, 614b26e3, 0adeafaa, f99f3...","[Carles Gil, Julian Gressel, Emanuel Reynoso, ...","[New England, D.C. United, Minnesota Utd, Real...","[49.1, 54.0, 49.9, 46.5, 51.0, 48.3, 47.1, 42....","[20024, 20526, 20887, 19279, 19332, 20501, 193...","[es ESP, de GER, ar ARG, sk SVK, am ARM, us US...","[MF, DF, MFFW, FWMF, FW, FWDF, DFMF, MFDF, DFF...","[25.4, 29.9, 26.5, 33.8, 29.1, 31.1, 29.4, 28....","[12.0, 12.2, 8.8, 7.2, 9.6, 7.5, 5.4, 9.9, 7.1...","[134, 96, 91, 87, 83, 80, 78, 75, 73, 69, 68, ...","[277.0, 166.0, 234.0, 145.0, 205.0, 179.0, 140...","[1240, 1116, 1377, 1234, 1252, 945, 1063, 1347...","[1669, 1732, 1618, 1647, 1654, 1655, 1336, 144...","[74.3, 64.4, 69, 83.6, 74.6, 75.6, 70.7, 73.4,...","[25809, 24290, 20799, 28368, 24667, 21280, 190...","[9931, 10948, 7764, 7668, 8427, 7300, 6522, 56...","[444, 415, 529, 553, 554, 669, 437, 499, 640, ...","[511, 523, 636, 605, 635, 771, 492, 596, 694, ...","[86.9, 79.3, 83.2, 91.4, 87.2, 86.8, 88.8, 83....","[535, 475, 385, 542, 398, 400, 304, 384, 544, ...","[652, 618, 522, 596, 505, 512, 386, 461, 613, ...","[82.1, 76.9, 73.8, 90.9, 78.8, 78.1, 83.3, 88....","[244, 222, 174, 263, 255, 144, 185, 154, 148, ...","[451, 530, 382, 409, 424, 270, 400, 329, 298, ...","[54.1, 41.9, 45.5, 64.3, 60.1, 53.3, 46.3, 46....","[1925, 2021, 2162, 1894, 1977, 2104, 1791, 182...","[7, 58, 5, 20, 17, 6, 22, 78, 42, 76, 28, 11, ...","[129, 378, 143, 241, 159, 258, 119, 205, 499, ...","[966, 916, 1016, 995, 1078, 1116, 818, 878, 77...","[1004, 834, 1177, 806, 840, 893, 995, 875, 673...","[102, 85, 83, 107, 108, 110, 104, 128, 52, 26,...","[1671, 1569, 1948, 1629, 1781, 1923, 1518, 161...","[66, 16, 107, 28, 20, 86, 53, 22, 57, 37, 72, ...","[98, 37, 193, 41, 32, 152, 147, 100, 88, 53, 1...","[67.3, 43.2, 55.4, 68.3, 62.5, 56.6, 58.5, 53,...","[67, 17, 112, 28, 21, 91, 64, 23, 59, 39, 74, ...","[8, 1, 17, 5, 2, 10, 3, 0, 4, 6, 7, 9]","[1438, 1055, 1668, 1288, 1310, 1594, 1341, 122...","[9798, 5443, 7466, 7702, 5979, 7119, 6627, 660...","[5239, 3503, 4081, 3905, 2654, 4527, 3121, 350...","[276, 171, 255, 210, 157, 273, 180, 177, 138, ...","[110, 38, 73, 74, 45, 82, 71, 61, 33, 55, 46, ...","[29, 17, 14, 28, 12, 24, 19, 30, 16, 4, 2, 36,...","[50, 24, 108, 37, 47, 92, 97, 77, 17, 31, 20, ...","[44, 25, 95, 31, 53, 75, 97, 42, 16, 67, 22, 8...","[1688, 1300, 1856, 1522, 1830, 1867, 1559, 164...","[1437, 1091, 1590, 1314, 1473, 1565, 1236, 131...","[85.1, 83.9, 85.7, 86.3, 80.5, 83.8, 79.3, 79....","[168, 113, 225, 157, 195, 252, 241, 101, 65, 2...","[0, 1, 2, 3]","[0, 1, 2]","[7, 35, 58, 23, 45, 60, 36, 31, 19, 62, 30, 40...","[76, 24, 107, 33, 68, 83, 69, 85, 26, 64, 57, ...","[0, 4, 2, 1, 5, 8, 3, 14, 26, 10, 7, 6, 22, 15...","[114, 144, 69, 46, 70, 59, 100, 62, 126, 41, 5...","[23, 68, 27, 25, 28, 30, 44, 53, 40, 21, 18, 3...","[4, 19, 43, 16, 26, 42, 20, 27, 39, 65, 24, 21...","[0, 1, 3, 2]","[0, 1, 2, 3]","[0, 1, 2]","[164, 251, 198, 203, 167, 178, 153, 163, 243, ...","[4, 30, 18, 10, 5, 6, 22, 26, 19, 8, 17, 15, 1...","[17.4, 44.8, 28.6, 12.9, 23.3, 27.8, 13.6, 44...."
type,object,object,object,float64,int64,object,object,object,float64,int64,float64,object,object,object,object,object,object,object,object,object,object,

In [ ]:
df.describe()

,Opposing_Poss,Opposing_Touches,xA,Key_pass,Comp_prog
count,730.000000,730.000000,730.000000,730.000000,730.000000
mean,49.974932,20231.846575,1.229589,11.979452,42.093151
std,3.206576,1036.339654,1.698503,15.947388,44.384491
min,42.600000,17449.000000,0.000000,0.000000,0.000000
25%,47.400000,19640.000000,0.100000,1.000000,6.000000
50%,50.200000,20180.000000,0.600000,6.000000,30.500000
75%,51.900000,21096.000000,1.700000,17.750000,62.000000
max,55.100000,22416.000000,12.200000,134.000000,277.000000


In [ ]:
df.fb_id.nunique()

730

In [ ]:
df.duplicated(subset=['fb_id']).sum()

0

In [ ]:
df.duplicated(subset=['fb_id']).value_counts()

False    730
True     195
dtype: int64

In [ ]:
dups = df[df.duplicated(subset=['fb_id'])]

In [ ]:
df.iloc[921:925]

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Nation,Position,Nineties,xA,Key_pass,Comp_prog,Total_Cmp,Total_Att,Total_Cmp_pct,Total_Tot_Dist,Total_Prg_Dist,Short_Cmp,Short_Att,Short_Cmp_pct,Medium_Cmp,Medium_Att,Medium_Cmp_pct,Long_Cmp,Long_Att,Long_Cmp_pct,touches,touches_def_box,touches_def_3rd,touches_mid_3rd,touches_att_3rd,touches_att_box,touch_live,dribble_Succ,dribble_Att,dribble_Succ_pct,num_dribble_past,dribble_megs,carries,carries_dist,carries_prog_dist,carries_prog,carries_att_3rd,carries_att_box,miss_control,dispossed,recep_targ,recep_succ,recept_pct,Yellow_cards,Red_cards,Second_yellow,Fouls,Fouls_drawn,Offsides,Crosses,def_interceptions,from_mish_tackles_won,PK_won,PK_conceded,OG,Recoveries,Aerial_Duels_lost,Aerial_win_pct
921,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,fr FRA,MFFW,1.1,0.0,0,2.0,32,51,62.7,468,96,18,21,85.7,12,16,75,1,11,9.1,61,1,13,23,28,5,56,1,2,50,1,0,38,201,73,4,2,0,1,1,45,37,82.2,6,0,0,1,0,0,3,2,1,0,0,0,9,2,66.7
922,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,fr FRA,MFFW,1.1,0.0,0,2.0,32,51,62.7,468,96,18,21,85.7,12,16,75,1,11,9.1,225,5,65,125,50,6,224,0,2,0,1,1,162,883,508,24,8,0,5,3,188,168,89.4,13,0,0,1,0,0,3,2,1,0,0,0,9,2,66.7
923,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,fr FRA,MFFW,1.1,0.0,0,2.0,32,51,62.7,468,96,18,21,85.7,12,16,75,1,11,9.1,61,1,13,23,28,5,56,1,2,50,1,0,38,201,73,4,2,0,1,1,45,37,82.2,6,0,0,7,5,0,3,9,3,0,0,0,25,6,75
924,ad454d78,Florian Valot,FC Cincinnati,51.0,20501,fr FRA,MFFW,1.1,0.0,0,2.0,32,51,62.7,468,96,18,21,85.7,12,16,75,1,11,9.1,225,5,65,125,50,6,224,0,2,0,1,1,162,883,508,24,8,0,5,3,188,168,89.4,13,0,0,7,5,0,3,9,3,0,0,0,25,6,75


In [ ]:
dups['player_name'].value_counts()

Mark-Anthony Kaye    15
Florian Valot        15
Florian Jungwirth    15
Nicolas Benezet      15
Liam Fraser          15
Jeremy Ebobisse      15
Brian White          15
Ramón Ábila          15
Sebastien Ibeagha    15
Dominique Badji      15
Amar Sejdic          15
Erik Hurtado         15
Corey Baird          15
Name: player_name, dtype: int64

In [ ]:
dups['player_name'].unique()

array(['Mark-Anthony Kaye', 'Nicolas Benezet', 'Brian White',
       'Erik Hurtado', 'Ramón Ábila', 'Jeremy Ebobisse', 'Florian Valot',
       'Corey Baird', 'Amar Sejdic', 'Florian Jungwirth',
       'Sebastien Ibeagha', 'Dominique Badji', 'Liam Fraser'],
      dtype=object)

In [ ]:
df_copy = df.copy()

In [ ]:
# Considering certain columns for dropping duplicates
df.drop_duplicates(subset=['player_name'],keep='first',inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 730 entries, 0 to 920
Data columns (total 63 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fb_id                  730 non-null    object 
 1   player_name            730 non-null    object 
 2   Squad_from_op          730 non-null    object 
 3   Opposing_Poss          730 non-null    float64
 4   Opposing_Touches       730 non-null    int64  
 5   Nation                 730 non-null    object 
 6   Position               730 non-null    object 
 7   Nineties               730 non-null    object 
 8   xA                     730 non-null    float64
 9   Key_pass               730 non-null    int64  
 10  Comp_prog              730 non-null    float64
 11  Total_Cmp              730 non-null    object 
 12  Total_Att              730 non-null    object 
 13  Total_Cmp_pct          728 non-null    object 
 14  Total_Tot_Dist         730 non-null    object 
 15  Total_

In [ ]:
df.duplicated(subset=['fb_id']).value_counts()

False    730
dtype: int64

use Tensorflow Data Validation to observe greater columns

In [ ]:

stats = tfdv.generate_statistics_from_dataframe(df)
 
tfdv.visualize_statistics(stats)

look for null values in dataframe

In [ ]:



  missing = [df.isnull().sum(),100*df.isnull().mean()]

  m = pd.DataFrame(missing)
  m = m.T
  m.columns=['count', '%']
  m = m.sort_values(by='count',ascending=False)
  m.head()

  m = pd.DataFrame(missing)
  m = m.T
  m.columns=['count','%']
  m =m.sort_values(by='count',ascending=False)
  m.head(10)

,count,%
dribble_Succ_pct,106.0,14.520548
Aerial_win_pct,90.0,12.328767
Long_Cmp_pct,24.0,3.287671
Short_Cmp_pct,5.0,0.684932
Medium_Cmp_pct,5.0,0.684932
Total_Cmp_pct,2.0,0.273973
recept_pct,2.0,0.273973
carries,0.0,0.000000
carries_dist,0.0,0.000000
carries_prog_dist,0.0,0.000000


look to drop players with nulls in features what are less than 1% missing 

look at a couple dribble succ_pct players and figure out why null and if mean is decent fill 

In [ ]:
df.dropna(subset=['Long_Cmp_pct','Short_Cmp_pct','Medium_Cmp_pct','Total_Cmp_pct','recept_pct'],inplace=True )
missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

m = pd.DataFrame(missing)
m = m.T
m.columns=['count','%']
m =m.sort_values(by='count',ascending=False)
m.head(10)

,count,%
dribble_Succ_pct,91.0,12.907801
Aerial_win_pct,80.0,11.347518
recept_pct,0.0,0.000000
num_dribble_past,0.0,0.000000
dribble_megs,0.0,0.000000
carries,0.0,0.000000
carries_dist,0.0,0.000000
carries_prog_dist,0.0,0.000000
carries_prog,0.0,0.000000
carries_att_3rd,0.0,0.000000


In [ ]:
df[['dribble_Succ_pct','Aerial_win_pct']].describe()

,dribble_Succ_pct,Aerial_win_pct
count,614,625
unique,197,283
top,100,50
freq,61,28


In [ ]:
df.Position.value_counts()

DF      227
MF      115
FWMF     96
FW       87
GK       65
MFFW     64
DFMF     18
MFDF     14
FWDF     10
DFFW      9
Name: Position, dtype: int64

In [ ]:
# state_price_means = ski_data.groupby('state')[['AdultWeekday', 'AdultWeekend']].mean()
# state_price_means.head()

# df.groupby('dummy')['returns'].agg(Mean='mean', Sum='sum')


fills =df[['Position','dribble_Succ_pct','Aerial_win_pct']]
fills[['dribble_Succ_pct','Aerial_win_pct']] = fills[['dribble_Succ_pct','Aerial_win_pct']].astype(float)
fills.info()
fills= fills.groupby('Position').agg({'dribble_Succ_pct': [np.mean, np.sum],'Aerial_win_pct': [np.mean, np.sum]})
fills.T.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 919
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Position          705 non-null    object 
 1   dribble_Succ_pct  614 non-null    float64
 2   Aerial_win_pct    625 non-null    float64
dtypes: float64(2), object(1)
memory usage: 42.0+ KB


/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


Position                         DF        DFFW  ...        MFDF         MFFW
dribble_Succ_pct mean     65.003431   57.100000  ...   64.015385    56.140323
                 sum   13260.700000  513.900000  ...  832.200000  3480.700000
Aerial_win_pct   mean     56.550909   40.688889  ...   50.850000    35.640323
                 sum   12441.200000  366.200000  ...  711.900000  2209.700000

[4 rows x 10 columns]

In [ ]:
#EDA

## look at one example 
# df[df['Squad_from_op']=='San Jose']


In [ ]:
df[['dribble_Succ_pct','Aerial_win_pct']] = df[['dribble_Succ_pct','Aerial_win_pct']].astype(float)

df['dribble_Succ_pct'].fillna((df['dribble_Succ_pct'].mean()), inplace=True)

In [ ]:


df['Aerial_win_pct'].fillna((df['Aerial_win_pct'].mean()), inplace=True)

In [ ]:
# Import Drive API and authenticate.
from google.colab import drive

# Mount your Drive to the Colab VM.
drive.mount('/gdrive')

# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/data/cleaned_player.csv', 'w') as f:
  df.to_csv(f)

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:

# Write the DataFrame to CSV file.
with open('/gdrive/My Drive/data/cleaned_player.csv', 'w') as f:
  df.to_csv(f)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 705 entries, 0 to 919
Data columns (total 63 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   fb_id                  705 non-null    object 
 1   player_name            705 non-null    object 
 2   Squad_from_op          705 non-null    object 
 3   Opposing_Poss          705 non-null    float64
 4   Opposing_Touches       705 non-null    int64  
 5   Nation                 705 non-null    object 
 6   Position               705 non-null    object 
 7   Nineties               705 non-null    object 
 8   xA                     705 non-null    float64
 9   Key_pass               705 non-null    int64  
 10  Comp_prog              705 non-null    float64
 11  Total_Cmp              705 non-null    object 
 12  Total_Att              705 non-null    object 
 13  Total_Cmp_pct          705 non-null    object 
 14  Total_Tot_Dist         705 non-null    object 
 15  Total_

use bar graphs to observe distriubtion of key features 